# <center>Search

# <font color='dd0000'>BFS

## [994. Rotting Oranges (easy)](https://leetcode-cn.com/problems/rotting-oranges/)

In a given grid, each cell can have one of three values:

- the value ```0``` representing an empty cell;
- the value ```1``` representing a fresh orange;
- the value ```2``` representing a rotten orange.

Every minute, any fresh orange that is adjacent (4-directionally) to a rotten orange becomes rotten.

Return the minimum number of minutes that must elapse until no cell has a fresh orange.  If this is impossible, return ```-1``` instead

In [ ]:
// BFS搜索，第一次将开始所有腐烂的橘子入队，每一轮将腐烂橘子的相邻位置入队，直到队空，最后判断是否还有好橘子

public int orangesRotting(int[][] grid) {
    Queue<Orange> q = new LinkedList<>();
    int m = grid.length;
    int n = grid[0].length;
    boolean noGoodOrange = true;  // 用来标记有没有好橘子
    for(int i = 0; i < m; i++){
        for(int j = 0; j < n; j++){
            if(grid[i][j] == 2){
                q.offer(new Orange(i, j));  //烂橘子入队
            } else if(grid[i][j] == 1 && noGoodOrange){  // 有好橘子,noGoodOrange置为false
                noGoodOrange = false;
            }
        }
    }
    if(q.isEmpty()){  // 队列为空有两种情况：1.全是好橘子,应返回-1；2.没有橘子,应返回0
        if(noGoodOrange) return 0; // noGoodOrange为true，说明没有好橘子，即没有橘子，返回0
        else return -1;            // noGoodOrange为false，说明有好橘子，那么一定全是好橘子,返回-1
    }
    int count = -1;  // 初始次数算第0次，所以从-1开始计数
    while(!q.isEmpty()){
        count++;
        int size = q.size();
        for(int k = 0; k < size; k++){
            Orange cur = q.poll();
            int i = cur.i;
            int j = cur.j;
            if(i - 1 >= 0 && grid[i - 1][j] == 1){ // 左边
                grid[i - 1][j] = 2;
                q.offer(new Orange(i - 1, j));
            }
            if(i + 1 < m && grid[i + 1][j] == 1){ // 右边
                grid[i + 1][j] = 2;
                q.offer(new Orange(i + 1, j));
            }
            if(j - 1 >= 0 && grid[i][j - 1] == 1){ // 上边
                grid[i][j - 1] = 2;
                q.offer(new Orange(i, j - 1));
            }
            if(j + 1 < n && grid[i][j + 1] == 1){ // 下边
                grid[i][j + 1] = 2;
                q.offer(new Orange(i, j + 1));
            }
        }
    }
    for(int i = 0; i < m; i++){     // 最后判断整个数组还有没有好橘子
        for(int j = 0; j < n; j++){
            if(grid[i][j] == 1)
                return -1;
        }
    }
    return count;
}

class Orange{
    int i;
    int j;

    public Orange(int i, int j){
        this.i = i;
        this.j = j;
    }
}

## [1091. Shortest Path in Binary Matrix (medium)](https://leetcode-cn.com/problems/shortest-path-in-binary-matrix/)

在一个 ```N × N``` 的方形网格中，每个单元格有两种状态：空（0）或者阻塞（1）。

一条从左上角到右下角、长度为 ```k``` 的畅通路径，由满足下述条件的单元格 ```C_1, C_2, ..., C_k``` 组成：

相邻单元格 ```C_i``` 和 ```C_{i+1}``` 在八个方向之一上连通（此时，```C_i``` 和 ```C_{i+1}``` 不同且共享边或角）

```C_1``` 位于 ```(0, 0)```（即，值为 ```grid[0][0]```）

```C_k``` 位于 ```(N-1, N-1)```（即，值为 ```grid[N-1][N-1]```）

如果 ```C_i``` 位于 ```(r, c)```，则 ```grid[r][c]``` 为空（即，```grid[r][c]``` == 0

返回这条从左上角到右下角的最短畅通路径的长度。如果不存在这样的路径，返回 -1 

In [ ]:
// 最短路径，用BFS

class Position{
    int i;
    int j;
    public Position(int i, int j){
        this.i = i;
        this.j = j;
    }
}

public int shortestPathBinaryMatrix(int[][] grid) {
    int n = grid.length;
    if(grid[0][0] == 1 || grid[n - 1][n - 1] == 1)
            return -1;
        
    boolean[][] isVisited = new boolean[n][n];  // 标记是否访问
    int[][] direction = {{-1,-1},{-1,0},{-1,1},{0,-1},{0,1},{1,-1},{1,0},{1,1}}; //!!!技巧：用来控制格子的方向
    Queue<Position> q = new LinkedList<>();
    q.offer(new Position(0, 0));
    isVisited[0][0] = true;
    int path = 0;
    while(!q.isEmpty()){
        path++;
        int size = q.size();
        for(int k = 0; k < size; k++){
            Position p = q.poll();
            int x = p.i;
            int y = p.j;
            if(x == n - 1 && y == n - 1) return path;
                
            for(int[] d : direction){
                int r = x + d[0];
                int c = y + d[1];
                if(r < 0 || r >= n || c < 0 || c >=n || grid[r][c] == 1 || isVisited[r][c])
                    continue;
                q.offer(new Position(r, c));
                isVisited[r][c] = true;
            }
        }
    }
    return -1;
}

## [279. Perfect Squares (medium)](https://leetcode-cn.com/problems/perfect-squares/)

Given a positive integer n, find the least number of perfect square numbers (for example, ```1, 4, 9, 16, ...```) which sum to n.

<font color='dd0000'>可以将每个整数看成图中的一个节点，**如果两个整数之差为一个平方数，那么这两个整数所在的节点就有一条边。**

要求解最小的平方数数量，就是**求解从节点 n 到节点 0 的最短路径**。(画图加深理解)</font>

In [ ]:
public int numSquares(int n) {
    List<Integer> squares = new ArrayList<>();
    int x = 1;
    while(x * x <= n){
        squares.add(x * x);
        x++;
    }
    
    Queue<Integer> q = new LinkedList<>();
    boolean[] isVisited = new boolean[n + 1];
    q.offer(n);
    isVisited[n] = true;
    int path = 0;
    while(!q.isEmpty()){
        path++;
        int size = q.size();
        for(int i = 0; i < size; i++){
            int cur = q.poll();
            for(int s : squares){
                int next = cur - s; // 当前数和完全平方数的差
                if(next == 0) return path; // 如果为0，说明已经找到n到0的最短路径
                if(next < 0) break;  // 如果小于0，那么就不必再和后面的完全平方数作差，跳出内循环
                if(isVisited[next]) continue;  // 如果大于0，当这个节点已经访问过，就继续循环，找下一个数
                q.offer(next);   // 新节点入队
                isVisited[next] = true;
            }
        }
    }
    return n;
}

## [1162. As Far from Land as Possible (medium)](https://leetcode-cn.com/problems/as-far-from-land-as-possible/)

Given an N x N ```grid``` containing only values ```0``` and ```1```, where ```0``` represents water and ```1``` represents land, find a water cell such that its distance to the nearest land cell is maximized and return the distance.

The distance used in this problem is the ***Manhattan distance***: the distance between two cells ```(x0, y0)``` and ```(x1, y1)``` is ```|x0 - x1| + |y0 - y1|```.

If no land or water exists in the ```grid```, return ```-1```

<font color='dd0000'>**理解题意：**要求的是，对每个0，找和它最近的1的距离，返回所有这些距离中的最大值 ===> 即：将所有1看作整体，逐层向外扩散，最后扩散到的0就是离所有陆地最远的0，也就是最大距离。因为每一层扩散，得到距离+1，所以扩散次数就等于距离。</font>(类似Rotting Oranges)

In [1]:
// BFS：!!!用大小为2的数组来表示点!!!

public int maxDistance(int[][] grid) {
    Queue<int[]> q = new LinkedList<>();
    int m = grid.length;
    int n = grid[0].length;
    int[][] directions = {{-1, 0}, {0, -1}, {0, 1}, {1, 0}};
    //首先将所有陆地入队
    for(int i = 0; i < m; i++){
        for(int j = 0; j < n; j++){
            if(grid[i][j] == 1){
                q.offer(new int[]{i, j});
            }
        }
    }
    int[] p = null; // 用来保存出队元素，同时可以用来标记是否全是海洋
    boolean hasOcean = false;  // 标记有没有海洋
    while(!q.isEmpty()){
        p = q.poll();
        for(int[] d : directions){
            int id = p[0] + d[0];
            int jd = p[1] + d[1];
            if(id < 0 || id >= m || jd < 0 || jd >= n || grid[id][jd] != 0)
                continue;
            q.offer(new int[]{id, jd});
            hasOcean = true;       // 遇到0了，说明有海洋
            grid[id][jd] = grid[i][j] + 1;  // 标记已访问，同时记录扩散次数
        }
    }
    if(p == null || !hasOcean) return -1;  // 全是海洋或全是陆地
    return grid[p[0]][p[1]] - 1;  // 因为起始是1，所以结果-1
}

## [127. Word Ladder (medium)](https://leetcode-cn.com/problems/word-ladder/)

Given two words (beginWord and endWord), and a dictionary's word list, find the length of shortest transformation sequence from beginWord to endWord, such that:

1. Only one letter can be changed at a time.
2. Each transformed word must exist in the word list.

**Note:**

- Return 0 if there is no such transformation sequence.
- All words have the same length.
- All words contain only lowercase alphabetic characters.
- You may assume no duplicates in the word list.
- You may assume beginWord and endWord are non-empty and are not the same.

<font color='dd0000'>**核心1：**判断转换出来的新单词是否在单词表的方法。从 $O(n \times l)$ 优化到 $O(26 \times l)$</font>

<font color='dd0000'>**核心2：**双向BFS。从 $O(n^k)$ 优化到 $O(n^{k/2})$</font>

In [6]:
// 最优解：双向bfs 用字母修改单词 查看字典里是否有修改后的单词
// （其他解参见提交记录）

public int ladderLength(String beginWord, String endWord, List<String> wordList) {
    if (wordList == null || !wordList.contains(endWord)) return 0; // 单词表为空或不含endWord则不可能转换，返回0
    
    wordList.add(beginWord); // 从两头搜索，所以把beginWord也加入单词表，极端情况在beginWord两头搜索相交    
    Set<String> dict = new HashSet<>(wordList); // 保存所有单词，用来判断转换后的词是否在单词表里
    
    Queue<String> q1 = new LinkedList<>();  // 队列1，从begin开始搜索
    Set<String> visited1 = new HashSet<>();  // 集合1，保存begin开始遍历过的单词
    Queue<String> q2 = new LinkedList<>(); // 队列2
    Set<String> visited2 = new HashSet<>(); // 集合2
    
    q1.offer(beginWord);
    visited1.add(beginWord);
    q2.offer(endWord);
    visited2.add(endWord);
    
    int count = 0;
    while(!q1.isEmpty() && !q2.isEmpty()){
        if(q2.size() < q1.size()){  // 每次选择节点少的队列搜索，这样能使搜索树更平衡，提高效率
            Queue<String> q = q1;
            q1 = q2;
            q2 = q;
            Set<String> s = visited1;
            visited1 = visited2;
            visited2 = s;
        }
        count++;
        int size = q1.size();
        for(int i = 0; i < size; i++){ // 通过逐位修改单词的每一个字母来进行转换，复杂度O(26*l)，l为单词长度
            String cur = q1.poll();
            char[] chars = cur.toCharArray();
            for(int j = 0; j < chars.length; j++){
                char t = chars[j];  // 把原字符暂存，用于恢复
                for(char c = 'a'; c <= 'z'; c++){
                    chars[j] = c;
                    String str = new String(chars); // 新转换出来的单词
                    if(visited1.contains(str)){  // 如果是本方向已访问过的，则跳过
                        continue;
                    }
                    if(visited2.contains(str)){  // 如果是另一个方向访问过的，说明搜索相交了，返回搜索次数+1（因为求的是长度）
                        return count + 1;
                    }
                    if(dict.contains(str)){ // 不符合上述三条，又是单词表中的词，则加入本方向已访问集合，并入队
                        q1.offer(str);
                        visited1.add(str);
                    }
                }
                chars[j] = t; // 把当前修改的字符还原，确保一次只修改一位
            }
        }
    }
    return 0;
}

## [542. 01 Matrix (medium)](https://leetcode-cn.com/problems/01-matrix/)

Given a matrix consists of 0 and 1, find the distance of the nearest 0 for each cell.

The distance between two adjacent cells is 1

<font color='dd0000'>

**方法一：** 多源BFS。和1162题类似，从所有的0开始扩散，每扩散到一层 1 ，和 0 的距离就加1

**方法二：** DP。从最近的0走到1，肯定只能从1周围上下左右4个点走到1，**第一次从左上角到右下角遍历整个表**，到表中任意位置i的时候，i上方和左方的位置已经遍历过了，所以可以判断从上方进入这个1和左方进入这个1的状况哪个最近，并在dp数组保存。同理，**第二次从右下角到左上角遍历整个表**，到i位置时，i右方和下方的位置状态已经更新过了，所以能判断从右边进入合算还是从下边进入合算，再加上第一次遍历保存的左方和上方的最优解就能判断出上下左右四个方向的最优解了

效率上 DP 更优
</font>

In [ ]:
// BFS
public int[][] updateMatrix(int[][] matrix) {
    int m = matrix.length;
    int n = matrix[0].length;
    Queue<int[]> q = new LinkedList<>();
    int[][] directions = {{-1, 0}, {0, -1}, {0, 1}, {1, 0}};
    boolean[][] visited = new int[m][n];
    for(int i = 0; i < m; i++){
        for(int j = 0; j < n; j++){
            if(matrix[i][j] == 0){
                q.offer(new int[]{i, j}); // 所有0入队
            }
        }
    }
    int[] cur = null;
    while(!q.isEmpty()){
        int size = q.size();
        for(int i = 0; i < size; i++){
            cur = q.poll();
            for(int[] d : directions){
                int id = cur[0] + d[0];
                int jd = cur[1] + d[1];
                if(id < 0 || id >= m || jd < 0 || jd >= n || visited[id][jd] || matrix[id][jd] != 1){
                    continue;
                }
                matrix[id][jd] = matrix[cur[0]][cur[1]] + 1; // 修改距离
                q.offer(new int[]{id, jd});
                visited[id][jd] = true;
            }
        }
    }
    return matrix;
}

// DP
public int[][] updateMatrix(int[][] matrix) {
    int m = matrix.length;
    int n = matrix[0].length;
    int[][] dp = new int[m][n];
    // 初始化
    for(int i = 0; i < m; i++){
        for(int j = 0; j < n; j++){
            if(matrix[i][j] == 1){
                dp[i][j] = Integer.MAX_VALUE - 1;  // 注意不能取最大数, 避免后面递推时+1溢出
            }
        }
    }
    // 从左上开始递推
    for(int i = 0; i < m; i++){
        for(int j = 0; j < n; j++){
            if(i - 1 >= 0){
                dp[i][j] = Math.min(dp[i][j], dp[i-1][j] + 1); // 从左边走
            }
            if(j - 1 >= 0){
                dp[i][j] = Math.min(dp[i][j], dp[i][j-1] + 1); // 从上边走
            }
        }
    }
    // 从右下开始递推
    for(int i = m - 1; i >= 0; i--){
        for(int j = n - 1; j >= 0; j--){
            if(i + 1 < m){
                dp[i][j] = Math.min(dp[i][j], dp[i+1][j] + 1); // 从下边走
            }
            if(j + 1 < n){
                dp[i][j] = Math.min(dp[i][j], dp[i][j+1] + 1); // 从右边走
            }
        }
    }
    return dp;
}

# <font color='dd0000'>DFS

## [200. Number of Islands (medium)](https://leetcode-cn.com/problems/number-of-islands/)

Given a 2d grid map of ```'1'```s (land) and ```'0'```s (water), count the number of islands. An island is surrounded by water and is formed by connecting adjacent lands horizontally or vertically. You may assume all four edges of the grid are all surrounded by water.

In [ ]:
//DFS:遍历数组，遇到'1'就开始以它为起点搜索，将和它相连的所有'1'通过DFS都置'0'(沉岛思想)，这样就能保证不重复统计岛数
// 直接修改原数组会破坏输入，不是太好，可以复制一份，或者通过isVisited数组来记录已访问过的点

int[][] directions = {{-1, 0}, {0, -1}, {0, 1}, {1, 0}};
public int numIslands(char[][] grid) {
    int count = 0;
    for(int i = 0; i < grid.length; i++){
        for(int j = 0; j < grid[0].length; j++){
            if(grid[i][j] == '1'){ // 遇到'1'说明这是一片岛，将和它连的所有'1'都破坏掉，避免再次访问
                count++;
                dfs(i, j, grid);
            }
        }
    }
    return count;
}

private void dfs(int i, int j, char[][] grid){
    if(i < 0 || i >= grid.length || j < 0 || j >= grid[i].length || grid[i][j] == '0')
        return;
    
    grid[i][j] = '0';
    for(int[] d : directions){ // 从(i, j)出发，四个方向搜索
        int id = i + d[0];
        int jd = j + d[1];
        dfs(id, jd, grid);
    }
}

<font color='dd0000'>⭐并查集的解法：</font>

In [ ]:
//并查集
public int numIslands(char[][] grid) {
    if(grid.length == 0){
        return 0;
    }
    UnionFind uf = new UnionFind(grid);
    int m = grid.length, n = grid[0].length;
    int[][] directions = {{-1, 0}, {0, -1}, {0, 1}, {1, 0}};
    for(int i = 0; i < m; i++){
        for(int j = 0; j < n; j++){
            if(grid[i][j] == '1'){
                for(int[] d : directions){
                    int id = i + d[0];
                    int jd = j + d[1];
                    if(id < 0 || id >= m || jd < 0 || jd >= n || grid[id][jd] == '0'){
                        continue;
                    }
                    uf.union(id * n + jd, i * n + j);
                }
            }
        }
    }
    return uf.count;
}

class UnionFind{
    int[] roots;
    int count = 0;

    public UnionFind(char[][] grid){
        int m = grid.length, n = grid[0].length;
        roots = new int[m * n];
        for(int i = 0; i < m; i++){
            for(int j = 0; j < n; j++){
                if(grid[i][j] == '1'){
                    roots[i * n + j] = i * n + j;
                    count++;
                }
            }
        }
    }

    public int findRoot(int i){ // 寻找指定节点的根节点
        int root = i;
        while(roots[root] != root){
            root = roots[root];
        }
        return root;
    }

    public void union(int p, int q){ // 合并两个集
        int pRoot = findRoot(p);
        int qRoot = findRoot(q);
        if(pRoot != qRoot){
            roots[qRoot] = pRoot;
            count--;
        }
    }
}

## [695. Max Area of Island (medium)](https://leetcode-cn.com/problems/max-area-of-island/)

Given a non-empty 2D array ```grid``` of ```0```'s and ```1```'s, an island is a group of ```1```'s (representing land) connected 4-directionally (horizontal or vertical.) You may assume all four edges of the grid are surrounded by water.

Find the maximum area of an island in the given 2D array. (If there is no island, the maximum area is 0.)

In [2]:
// DFS：和上题一样，遇到1就进去搜索，不同点在于这次在DFS搜索过程中要记录1的数量（面积）

int[][] directions = {{-1, 0}, {0, -1}, {0, 1}, {1, 0}};
public int maxAreaOfIsland(int[][] grid) {
    int ans = 0;
    int area = 0;
    for(int i = 0; i < grid.length; i++){
        for(int j = 0; j < grid[i].length; j++){
            if(grid[i][j] == 1){
                area = dfs(i ,j, grid);
                ans = Math.max(ans, area); // 结果不断取最大值
            }
        }
    }
    return ans;
}

private int dfs(int i, int j, int[][] grid){  // 返回值是1的个数
    if(i < 0 || i >= grid.length || j < 0 || j >= grid[i].length || grid[i][j] == 0)
        return 0;
    
    int area = 1; // 自己本身面积是1
    grid[i][j] = 0; // 计算过了就把1置0，避免重复访问
    for(int[] d : directions){
        int id = i + d[0];
        int jd = j + d[1];
        area += dfs(id, jd, grid);
    }
    return area;
}

In [4]:
// 也可以用BFS，但效率比较低
public int maxAreaOfIsland2(int[][] grid) {
    if(grid.length < 1) return 0;
    
    Queue<Point> q = new LinkedList<>();
    int m = grid.length;
    int n = grid.length;
    boolean[][] isVisited = new boolean[m][n];
    int[][] direction = {{-1, 0}, {0, -1}, {0, 1}, {1, 0}};
    int ans = 0;
    int area = 0;
    for(int i = 0; i < m; i++){
        for(int j = 0; j < n; j++){
            if(grid[i][j] == 1 && !isVisited[i][j]){  // 遇到没有访问过的1，就进入搜索
                q.offer(new Point(i , j));
                while(!q.isEmpty()){
                    int size = q.size();
                    for(int k = 0; k < size; k++){
                        Point cur = q.poll();
                        area++;  // 每访问一个，面积+1
                        int x = cur.x;
                        int y = cur.y;
                        for(int[] d : direction){
                            int xd = x + d[0];
                            int yd = y + d[1];
                            if(xd < 0 || xd >= m || yd < 0 || yd >= n || grid[xd][yd] == 0 || isVisited[xd][yd])
                                continue;
                            q.offer(new Point(xd, yd)); // 将周围一层合法的岛入队
                            isVisited[xd][yd] = true;
                        }
                    }
                }
            }
            ans = Math.max(ans, area);
            area = 0;
        }
    }
    return ans;
}

class Point{
    int x;
    int y;
    public Point(int x, int y){
        this.x = x;
        this.y = y;
    }
}

## [剑指Offer13. 机器人的运动范围 (medium)](https://leetcode-cn.com/problems/ji-qi-ren-de-yun-dong-fan-wei-lcof/)

地上有一个m行n列的方格，从坐标 ```[0,0]``` 到坐标 ```[m-1,n-1]``` 。一个机器人从坐标 ```[0, 0]``` 的格子开始移动，它每次可以向左、右、上、下移动一格（不能移动到方格外），也不能进入行坐标和列坐标的数位之和大于k的格子。例如，当k为18时，机器人能够进入方格 ```[35, 37]``` ，因为3+5+3+7=18。但它不能进入方格 ```[35, 38]```，因为3+5+3+8=19。请问该机器人能够到达多少个格子？

<font color='dd0000'>注意：规定了从```[0,0]```开始移动，所以从起点开始搜索，直到走不动即可。即使有其他符合条件的坐标，但是从起点无法走过去，所以不计入结果。**并且由于一开始是从左上角出发，可以只对【右和下】两个方向进行搜索**。</font>

In [ ]:
boolean[][] isVisited;
public int movingCount(int m, int n, int k) {
    isVisited = new boolean[m][n];
    int res = dfs(0, 0, k, m, n);
    return res;
}

private int dfs(int i, int j, int k, int m, int n){
    if(i < 0 || i >= m || j < 0 || j >= n || !check(i, j, k) || isVisited[i][j]){
        return 0;
    }
    isVisited[i][j] = true;
    return 1 + dfs(i, j + 1, k, m, n) + dfs(i + 1, j, k, m, n);
}

private boolean check(int i, int j, int k){
    int sum = 0;
    while(i > 0){
        sum += i % 10;
        i / = 10;
    }
    while(j > 0){
        sum += j % 10;
        j /= 10;
    }
    return sum <= k;
}

## [22. Generate Parentheses (medium)](https://leetcode-cn.com/problems/generate-parentheses/)

Given n pairs of parentheses, write a function to generate all combinations of well-formed parentheses.

<font color='dd0000'>⭐**DFS：画搜索树辅助理解**</font>

<font color='dd0000'>
    
1. 当前左右括号都有大于 00 个可以使用的时候，才产生分支；

2. 产生左分支的时候，只看当前是否还有左括号可以使用；

3. 产生右分支的时候，还受到左分支的限制，右边剩余可以使用的括号数量一定得在严格大于左边剩余的数量的时候，才可以产生分支；

4. 在左边和右边剩余的括号数都等于 00 的时候结算。
</font>

In [1]:
public List<String> generateParenthesis(int n) {
    List<String> res = new ArrayList<>();
    dfs(n, n, "", res);  // 入口，传入总数量和空串
    return res;
}

private void dfs(int left, int right, String s, List<String> res){ //left、right表示左右括号剩余数量，s表示连接的字符串
    if(left == 0 && right == 0){  // 左右括号都用完了，说明连接成了一个结果
        res.add(s);
        return;
    }
    if(left > right){ // 左括号数量大于右括号数量，说明先连接了右括号，无法匹配，为无效结果，直接返回
        return;
    }
    if(left > 0){
        dfs(left - 1, right, s + "(", res); // 连接左括号
    } 
    if(right > 0){
        dfs(left, right - 1, s + ")", res); // 连接有括号
    }
}

<font color='dd0000'>**回溯：**「回溯算法」强调了在状态空间特别大的时候，只用一份状态变量去搜索所有可能的状态，在搜索到符合条件的解的时候，通常会做一个拷贝，这就是为什么经常在递归终止条件的时候，有```res.add(new ArrayList<>(path));```这样的代码。正是因为全程使用一份状态变量，因此它就有「恢复现场」和「撤销选择」的需要。</font>

In [4]:
// 回溯（有点丑陋）

List<String> res = new ArrayList<>();
boolean[] flag;  // 用来标记哪个位置是左括号(true)哪个位置是右括号(false)

public List<String> generateParenthesis(int n) {
    flag = new boolean[n * 2];
    flag[0] = true;   // 第一个位置一定为左括号
    helper(1, n, 1);
    return res;
}

private boolean isValid() {  // 判断生成的括号是否有效：即左右括号能匹配
    int left = 0;
    int right = 0;
    for (int i = 0; i < flag.length; i++) {
        if (flag[i])
            left++;
        else
            right++;
        if (right > left)
            return false;
    }
    return true;
}

private void helper(int i, int n, int count) { // i表示当前要生成左括号的位置，n表示数量，count表示已经生成的数量
    if (count == n) {  // 如果生成的左括号已经达到最大数量，那么就生成字符串，但要先判断当前所标记出来的是否有效
        if (!isValid())
            return;
        StringBuilder sb = new StringBuilder();
        for (boolean b : flag) {
            sb.append(b ? "(" : ")");
        }
        res.add(sb.toString());
        return;
    }

    for (int j = i; j < flag.length - 1; j++) { // 每个位置依次填入左括号
        if (!flag[j]) {
            flag[j] = true;
            helper(j + 1, n, count + 1);
            flag[j] = false;   // 撤销选择，恢复现场
        }
    }
}